In [1]:
from ivisual import *
import numpy as np
from numpy import linalg as LA
import itertools

############ Load the atom positions and spin directions.(only read limited some atoms in order in the filed)
###############################################################

def load_spins(fpath, nbatom, incNbCell):
    
    atpos=np.zeros([nbatom,3]) #store atom positons in a unit cell
    allpos=np.zeros([nbatom*6**3,3])#store atom positons in a super cell
    spins =np.zeros([nbatom*6**3,3])#store spins in a super cell
    #read data
    with open(fpath, 'rU') as f:
        i=0
        for line in f:
            line=line.rstrip('\n').split()# without giving delimenter, it will treat conseutive space as one
            if line[0]=="SITE":
                atpos[i,:]=line[1:] #automatically convert string to np.array
                i=i+1
                
    with open(fpath, 'rU') as f:
        i=0        
        for line in f:        
            line=line.rstrip('\n').split()# without giving delimenter, it will treat conseutive space as one
            if line[0]=="SPIN":
                allpos[i,:]=np.array(line[2:5],dtype='f8')+atpos[np.int(line[1])-1,:]
                spins[i,:]=line[5:8]
                i=i+1
                if i==incNbCell*nbatom:
                    break
                
    ##### find the nearest neighers
    bonds = np.zeros([5,1000])#store the bonds in the unitCell: atom1,atom2,pos[a2]-pos[a1]
    idx = 0
     #len(allpos)-1
    for i in np.arange(0,incNbCell*nbatom,1):
        for ii in np.arange(i+1,incNbCell*nbatom,1):
            distance = LA.norm(allpos[i]-allpos[ii])
            if distance==2*np.sqrt(3/64.) and ~np.in1d(ii,bonds[0,np.where(bonds[1,:]==i)])[0]:
                bonds[:,idx]=np.append(np.array([i, ii]),allpos[ii]-allpos[i])
                idx +=1
    
    return atpos, allpos, spins, bonds
    

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
################ Read file 
# Diamond lattice
fpath= r'D:\9_Program files\SpinVert\gdzro_ill\gd_ill_60mk-20k_with_paramag\diamond_correlations\spinice_spins_01_diamond.txt'
incNbCell = 5
nbatom = 8
atpos, allpos, spins, bonds = load_spins(fpath,nbatom,incNbCell)
# Pyrochlore lattice
fpath= r'D:\9_Program files\SpinVert\gdzro_ill\gd_ill_60mk-20k_with_paramag\diamond_correlations\spinice_spins_01.txt'
nbatom0=16
atpos0, allpos0, spins0, _ = load_spins(fpath,nbatom0,incNbCell)



# #############################
scene1 = display(title='MagStr', width=500, height=500,x=500,y=500, center=(0.5,0.5,1), userzoom = True,
                background=color.black,exit=False)

#scene1.autocenter = True
#scene1.autoscale = True

#scene1.lights = []
#scene1.ambient= 2

spinl = 0.5; spinl0 = 0.2
atomr = 0.5; cylr = 0.005

for i in range(bonds.shape[1]):
    pointer = cylinder(pos=allpos[bonds[0,i]], axis=bonds[2:,i], radius=cylr, materials=materials.wood)

#for i in range(len(allpos)):
for i in range(incNbCell*nbatom):
    pointer = arrow(pos=allpos[i]-spinl*spins[i]/2, axis=spinl*spins[i],color=color.yellow) # spins
    pointer = sphere(pos=allpos[i], color=color.red, radius=atomr*0.1) # atoms
    #pointer = label(pos=allpos[i]+(0,0,0.1), text=str(i%16))  
       
mybox = box(pos=(0.5,0.5,0.5), length=1, height=1, width=1, opacity=0.5)

for i in range(incNbCell*nbatom0):
    pointer = arrow(pos=allpos0[i]-spinl0*spins0[i]/2, axis=spinl0*spins0[i],color=color.blue) # spins

# the Sx    
##spins0x = np.zeros_like(spins0)
##spins0x[:,0] = spins0[:,0]
##
##for i in range(incNbCell*nbatom0):
##    pointer = arrow(pos=allpos0[i]-spinl0*spins0x[i]/2, axis=spinl0*spins0x[i]) # spins


C:\Users\kew\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:28: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [9]:
# Try to draw polyhedras but failed
from scipy.spatial import Delaunay
tri = Delaunay(allpos0)
#print allpos0, list(tri)
print tri.simplices, allpos0[tri.simplices].shape

[[ 0 13  4  8]
 [ 9  1  5 12]
 [26 22 18 34]
 ..., 
 [56 52 78 77]
 [56 75 78 64]
 [56 75 69 64]] (309L, 4L, 3L)


In [15]:
#Polyhedron(allpos0[tri.simplices][0,:])
scene1 = display(title='MagStr', width=500, height=500,x=500,y=500, center=(0.5,0.5,1), userzoom = True,
                background=color.black,exit=False)
Tetrahedron().draw()

In [2]:
"""
Some infrastructure for working with Vectors and Edges, including
an xyplotter generator and axes maker.

By Kirby Urner, Sept 13, 2006

Updated Sept 29, 2006:
make Edge color a class-level attribute
add funky derivative demo
refactor a bit

Code:
http://www.4dsolutions.net/ocn/python/stickworks.py

For colorized source:
http://www.4dsolutions.net/cgi-bin/py2html.cgi?script=/ocn/python/stickworks.py

Some relevant discussion:
http://mail.python.org/pipermail/edu-sig/2006-September/007145.html
http://mail.python.org/pipermail/edu-sig/2006-September/007149.html
http://mail.python.org/pipermail/edu-sig/2006-September/007150.html
http://mail.python.org/pipermail/edu-sig/2006-September/007312.html
"""

from ivisual import vector, cylinder, cross, dot, diff_angle
import ivisual

class Vector (object):

    """
    A wrapper for visual.vector that expresses a cylinder via draw(),
    always pegged to the origin
    """

    radius = 0.03
    
    def __init__(self, xyz, color=(0,0,1)):
        self.v = vector(*xyz)
        self.xyz = xyz
        self.color = color
        self.cyl = None

    def draw(self):
        """define and render the cylinder"""        
        self.cyl = cylinder(pos = (0,0,0), axis = self.v, radius = self.radius, color = self.color)

    def erase(self):
        """toss the cylinder"""        
        if self.cyl:
            self.cyl.visible = 0
        self.cyl = None

    def __repr__(self):
        return 'Vector @ (%s,%s,%s)' % self.xyz

    # some vector ops, including scalar multiplication
    
    def diff_angle(self, other):
        return self.v.diff_angle(other.v)

    def cross(self, other):
        temp = cross(self.v, other.v)
        return Vector((temp.x, temp.y, temp.z))
    
    def dot(self, other):
        return dot(self.v, other.v)

    def __sub__(self, other):
        temp = self.v - other.v
        return Vector((temp.x, temp.y, temp.z))

    def __add__(self, other):
        temp = self.v + other.v
        return Vector((temp.x, temp.y, temp.z))

    def __mul__(self, scalar):
        temp = self.v * scalar
        return Vector((temp.x, temp.y, temp.z))

    __rmul__ = __mul__

    def __neg__(self):
        return Vector((-self.v.x, -self.v.y, -self.v.z))

    def _length(self):
        return pow(self.v.x ** 2 + self.v.y ** 2 + self.v.z ** 2, 0.5)

    length = property(_length)
    
class Edge (object):

    """
    Edges are defined by two Vectors (above) and express as cylinder via draw().
    """

    radius = 0.03
    color = (1,0,0)

    def __init__(self, v0, v1, color=None):
        if not color==None:
            self.color = color
        self.v0 = v0
        self.v1 = v1
        self.cyl = None        

    def draw(self):
        """define and render the cylinder"""
        temp = (self.v1 - self.v0).xyz
        self.cyl = cylinder(pos = self.v0.xyz, axis = vector(*temp), 
                              radius = self.radius, color = self.color)

    def erase(self):
        """toss the cylinder"""
        if self.cyl:
            self.cyl.visible = 0
        self.cyl = None
        
    def __repr__(self):
        return 'Edge from %s to %s' % (self.v0, self.v1)


def xyplotter(domain, f):
    """
    domain should be an initialized generator, ready for next() triggering.
    f is any function of x.  Consecutive Vectors trace connected edges.
    """
    x0 = domain.next()    
    y0  = f(x0)        
    while True:
        x1 = domain.next()
        y1 =  f(x1)
        e = Edge( Vector((x0, y0, 0)), Vector((x1, y1, 0)) )
        e.draw()
        yield None
        x0, y0 = x1, y1
        
def axes(x=0,y=0,z=0):
    """
    Draw some axes on the VPython canvas
    """
    v0 = Vector((x,0,0))
    v0.draw()
    v0 = Vector((-x,0,0))
    v0.draw()

    v0 = Vector((0,y,0))
    v0.draw()
    v0 = Vector((0,-y,0))
    v0.draw()

    v0 = Vector((0,0,z))
    v0.draw()
    v0 = Vector((0,0,-z))
    v0.draw()

    
def dgen(start, step):
    """
    generic domain generator
    """
    while True:
        yield start
        start += step

def testme():
    """
    >>> from stickworks import testme
    Visual 2005-01-08
    >>> testme()

    See:
    http://www.4dsolutions.net/ocn/graphics/cosines.png
    """
    
    from math import cos

    def f(x):  return cos(x)

    d = dgen(-5, 0.1)
    axes(-5,1,0)
    graph = xyplotter(d, f)

    for i in xrange(100):
        graph.next()

def testmemore():
    """
    See:
    http://www.4dsolutions.net/ocn/graphics/pycalculus.png
    """

    def snakeywakey(x):
        """
        Polynomial with x-axis crossings at 3,2,-3,-7, with scaler
        to keep y-values under control (from a plotting point of view)
        """
        return 0.01 * (x-3)*(x-2)*(x+3)*(x+7)

    def deriv(f, h=1e-5):
        """
        Generic df(x)/dx approximator (discrete h)
        """       
        def funk(x):
            return (f(x+h)-f(x))/h
        return funk
    

    d1 = dgen(-8, 0.1)
    d2 = dgen(-8, 0.1)
    d3 = dgen(-8, 0.1)
    
    axes(-8,5,3)

    deriv_snakeywakey = deriv(snakeywakey)
    second_deriv = deriv(deriv_snakeywakey)
    
    graph1 = xyplotter(d1, snakeywakey)
    graph2 = xyplotter(d2, deriv_snakeywakey)
    graph3 = xyplotter(d3, second_deriv)    

    Edge.color = (1,0,0)  # make snakeywakey red
    
    for i in xrange(130):
        graph1.next()

    Edge.color = (0,1,0)  # make derivative green
    
    for i in xrange(130):
        graph2.next()

    Edge.color = (0,1,1)  # make 2nd derivative cyan

    for i in xrange(130):
        graph3.next()

if __name__ == '__main__':
    testmemore()


In [4]:
"""
Kirby Urner
4D Solutions
First published: Apr 29 2007

Suitable for spatial geometry and/or synergetics students.

Update May 10:  I'd forgotten 8 of the 20 Icosahedron triangles!  Added.
Update May 13:  Added Octahedron, Mite, Coupler

"""

#from stickworks import Vector, Edge
from ivisual import color
#from ivisual import * # by Xu
from math import sqrt

phi = (sqrt(5) + 1)/2.0
        
class Polyhedron (object):

    # defaults may be overridden

    showfaces      = True
    showedges      = True
    showvertices   = True
    # default POV-Ray textures

    face_texture   = 'T_Stone14'   # from stones.inc

    vertex_texture = 'T_Silver_1A' # from metals.inc

    edge_texture   = 'T_Copper_4A' # from metals.inc

    
    def scale(self, scalefactor):
        newverts = {}
        for v in self.vertices:
            newverts[v] = self.vertices[v] * scalefactor
        return self.__class__(newverts)

    __mul__ = __rmul__ = scale

    def translate(self, vector):
        newverts = {}
        for v in self.vertices:
            newverts[v] = self.vertices[v] + vector
        return self.__class__(newverts)

    __add__ = __radd__ = translate
    
    def _distill(self):

        edges = []
        unique = set()
        
        for f in self.faces:
            for pair in zip(f , f[1:] + (f[0],)):
                unique.add( tuple(sorted(pair)) )

        for edge in unique:
            edges.append( Edge(self.vertices[edge[0]],self.vertices[edge[1]]) )

        return edges            

    def draw(self):
        # VPython wireframe view, native to stickworks.py

        for e in self.edges:
            e.draw()
            

class Amodule (Polyhedron) :
    pass

class Bmodule (Polyhedron) :
    pass

class Mite (Polyhedron) :

    def __init__(self,
                 verts  = dict(j = Vector(( 0,  1, 0)),
                               o = Vector(( 0,  0, 0)),
                               r = Vector(( 1,  0, 1)),
                               s = Vector(( 1,  0,-1)))):
                 
        # 4 vertices

        self.vertices = verts

        # 4 faces

        self.faces = (('j','o','r'),('j','r','s'),('j','s','o'),('o','r','s'))

        self.edges = self._distill()
        
class Smite (Polyhedron) :
    pass

class Coupler (Polyhedron) :
    
    def __init__(self,
                 verts  = dict(j = Vector(( 0,  1, 0)),
                               l = Vector(( 0, -1, 0)),
                               q = Vector((-1,  0, 1)),
                               r = Vector(( 1,  0, 1)),
                               s = Vector(( 1,  0,-1)),
                               t = Vector((-1,  0,-1)))):

        # 6 vertices

        self.vertices = verts

        # 8 faces

        self.faces = (('j','q','r'),('j','r','s'),('j','s','t'),('j','t','q'),
                      ('l','q','r'),('l','r','s'),('l','s','t'),('l','t','q'))

        self.edges = self._distill()

class Tetrahedron (Polyhedron) :

    def __init__(self,
                 verts  = dict(a = Vector((-1, -1, 1)),
                               b = Vector((-1,  1, -1)),
                               c = Vector((1, 1, 1)),
                               d = Vector((1, -1, -1)))):
        """
        Imagine a cube centered at the origin and with
        a positive octant vertex at (1,1,1).  Inscribe
        a regular tetrahedron as six face diagonals therein.
        """
        # 4 vertices

        self.vertices = verts

        # 4 faces

        self.faces = (('a','b','c'),('a','c','d'),
                      ('a','d','b'),('b','d','c'))

        self.edges = self._distill()

class Cube (Polyhedron):

    def __init__(self, verts = dict( a = Vector((-1, -1, 1)),
                                     b = Vector((-1,  1, -1)),
                                     c = Vector((1, 1, 1)),
                                     d = Vector((1, -1, -1)),
                                     e = Vector((1,  1, -1)),
                                     f = Vector((1, -1,  1)),
                                     g = Vector((-1, -1, -1)),
                                     h = Vector((-1, 1, 1)))):

        # 8 vertices

        self.vertices = verts

        # 6 faces

        self.faces = (('a','f','c','h'),('h','c','e','b'),
                      ('b','e','d','g'),('g','d','f','a'),
                      ('c','f','d','e'),('a','h','b','g'))

        self.edges = self._distill()

class Octahedron (Polyhedron):

    def __init__(self, verts = dict( i = Vector(( 0, 0, 1)),
                                     j = Vector(( 0, 1, 0)),
                                     k = Vector(( 0, 0,-1)),
                                     l = Vector(( 0,-1, 0)),
                                     m = Vector(( 1, 0, 0)),
                                     n = Vector((-1, 0, 0)))):

        # 6 vertices

        self.vertices = verts

        # 8 faces

        self.faces = (('i','l','m'),('i','m','j'),('i','j','n'),('i','n','l'),                      
                      ('k','l','m'),('k','m','j'),('k','j','n'),('k','n','l'))

        self.edges = self._distill()
                                     
class Dodecahedron (Polyhedron):
    pass

class Icosahedron (Polyhedron):

    def __init__(self, verts = dict(
            # 12 vertices at the corners of 3 mutually

            # orthogonal golden rectangles

            xya=Vector(( phi/2, 0.5, 0.0)), # phi rectangle in xy

            xyb=Vector(( phi/2,-0.5, 0.0)),
            xyc=Vector((-phi/2,-0.5, 0.0)),
            xyd=Vector((-phi/2, 0.5, 0.0)),
            #-----------------------------

            xza=Vector((-0.5, 0.0, phi/2)), # Phi rectangle in xz 

            xzb=Vector(( 0.5, 0.0, phi/2)),
            xzc=Vector(( 0.5, 0.0,-phi/2)),
            xzd=Vector((-0.5, 0.0,-phi/2)),
            #-----------------------------

            yza=Vector(( 0.0, phi/2, 0.5)), # Phi rectangle in yz 

            yzb=Vector(( 0.0, phi/2,-0.5)),
            yzc=Vector(( 0.0,-phi/2,-0.5)),
            yzd=Vector(( 0.0,-phi/2, 0.5)),
            )):

        # 12 vertices

        self.vertices = verts

        # 20 equiangular triangles

        self.faces = (
            ('xza','xzb','yzd'),
            ('yzd','xzb','xyb'),
            ('xyb','xzb','xya'),
            ('xya','yza','xzb'),
            ('xzb','yza','xza'),

            ('xzd','xzc','yzb'),
            ('yzb','xzd','xyd'),
            ('xyd','xzd','xyc'),
            ('xyc','xzd','yzc'),
            ('yzc','xzd','xzc'),

            ('xyd','yzb','yza'),
            ('yza','yzb','xya'),
            ('xya','yzb','xzc'),
            ('xzc','xya','xyb'),
            ('xyb','xzc','yzc'),
            ('yzc','xyb','yzd'),
            ('yzd','yzc','xyc'),
            ('xyc','yzd','xza'),
            ('xza','xyc','xyd'),
            ('xyd','xza','yza')
            )
           

        self.edges = self._distill()
        
        self.rectangles = (
            ('xya','xyb','xyc','xyd'),
            ('xza','xzb','xzc','xzd'),
            ('yza','yzb','yzc','yzd'))    

    def goldrects(self):
        Edge.color = green
        for r in self.rectangles:
            c0,c1,c2,c3 = [self.vertices[i] for i in r]
            Edge(c0,c1).draw()
            Edge(c1,c2).draw()
            Edge(c2,c3).draw()
            Edge(c3,c0).draw()

class Cuboctahedron (Polyhedron):
    pass

def test():
    """
    The Concentric Hierarchy by R. Buckminster Fuller
    """
    Edge.color = color.orange
    tetra = Tetrahedron() * 0.5
    tetra.draw()
    
    Edge.color = color.green
    cube = Cube() * 0.5
    cube.draw()

    Edge.color = color.red
    cube = Octahedron()
    cube.draw()

    Edge.color = color.cyan
    ico = Icosahedron() * sqrt(2)
    ico.draw()

def test2():
    """
    Coupler in a Cube (canonical volumes 1 and 3 respectively)
    """

    Edge.color = color.orange
    tetra = Tetrahedron()
    tetra.draw()
    
    Edge.color = color.blue
    coupler = Mite()
    coupler.draw()

    #Edge.color = color.blue

    #coupler = Coupler()

    #coupler.draw()

    
    Edge.color = color.green
    cube = Cube()
    cube.draw()    

if __name__ == '__main__':
    test()
    # test2()

In [ ]:
Ployhetron()